# Большая домашка 1: классификация отзывов

## Данные

Скачиваются по [ссылке](https://drive.google.com/open?id=1gJEpwM3zdV5xe9nRU0VwQmGTiOTQdp10). Это отзывы пользователей на фильмы с КиноПоиска.

In [2]:
!pip install pandas

  Using cached pandas-1.0.3-cp36-cp36m-manylinux1_x86_64.whl (10.0 MB)


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('reviews.csv')

In [5]:
df.head()

,label,text
0,1,Эпиграф Добро которое ты делаешь от сердца ты ...
1,1,Теперь это один из моих любимых фильмов в жанр...
2,1,Что скрыто в фильме Лучше не бывает Одна шикар...
3,1,Перед нами очень милое и доброе кино которое л...
4,1,Завязка Мелвин Удал популярный писатель Нет не...


In [6]:
set(df.label)

{-1, 0, 1}

В колонке `label` три класса: положительный (1), нейтральный (0) и отрицательный (-1).

## Baseline

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
vec = CountVectorizer()
bag_of_words = vec.fit_transform(df.text)
X_train, X_test, y_train, y_test = train_test_split(bag_of_words, df.label)

In [9]:
nb = MultinomialNB()
clf = nb.fit(X_train, y_train)

In [10]:
from sklearn.metrics import classification_report

In [11]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.67      0.20      0.31      1085
           0       0.31      0.08      0.12      1349
           1       0.82      0.98      0.89      9192

    accuracy                           0.80     11626
   macro avg       0.60      0.42      0.44     11626
weighted avg       0.75      0.80      0.75     11626



## Задание

Побейте baseline выше как минимум тремя способами.


**Обязательно**:
* попробовать разный препроцессинг (2 балла)
* попробовать разные классификаторы (2 балла)
* поподбирать гиперпараметры классификаторов, можно с использованием GreedSearch (2 балла)
* попробуйте объяснить полученные результаты (2 балла)
* визуализируйте веса признаков для лучшей модели (если лучшая модель не выдаёт коэффициенты признаков, возьмите логистическую регрессию); какие признаки оказались самыми значимыми? (2 балла)

Если вы перебрали уже много разных способов и выполнили все обязательные критерии, но побить baseline тремя способами так и не получилось, не беспокойтесь. Главное -- полнота попыток :)

Good luck and have fun! ;)

### Бонус
(+2 балла к основному заданию; оценка за всё задание может быть выше 10)

Придумайте ещё 2-3 осмысленных споцоба визуализации данных или результатов. Добавьте их ниже или в текст основного задания. и сошлитесь на них здесь.

### Как это сделать с w2v

In [11]:
!cp 6/187.zip .

In [12]:
!unzip 187.zip

Archive:  187.zip
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [20]:
from gensim.models.keyedvectors import FastTextKeyedVectors
from nltk.tokenize import word_tokenize

In [13]:
m = FastTextKeyedVectors.load('model.model')

In [21]:
def embed_text(text):
    tokens = word_tokenize(text)
    word_embeddings = []
    for token in tokens:
        try:
            word_embeddings.append(m[token])
        except AttributeError:
#             print('problem with word', token)
            pass

    text_embedding = sum(word_embeddings) / len(word_embeddings)
    return text_embedding

In [22]:
embeddings = [embed_text(text) for text in df.text]

In [30]:
len(embeddings)

46501

In [31]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, df.label)

In [33]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

/home/maryszmary/.local/share/virtualenvs/DPO_2020-hd13euOw/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

          -1       0.58      0.46      0.51      1083
           0       0.44      0.08      0.14      1442
           1       0.84      0.97      0.90      9101

    accuracy                           0.82     11626
   macro avg       0.62      0.50      0.52     11626
weighted avg       0.77      0.82      0.77     11626

